In [1]:
cd /home/dev/2025/transaction-generation

/home/dev/2025/transaction-generation


/usr/local/lib/python3.10/dist-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
from notebooks.exps.config_factory import load_base_yaml, generate_configs, str_to_df, collect_res, df_to_str, compare_yamls

In [3]:
import pandas as pd
def str_to_df(s, sep='\t'):
    s = s.strip().split("\n")
    df = pd.DataFrame(columns=s[0].split(" ++ "))
    for i, line in enumerate(s[1:]):
        line = line.split(" ++ ")
        df.loc[i] = line
    for i in range(df.shape[0]):
        for j in range(df.shape[1]):
            try:
                df.iloc[i, j] = eval(df.iloc[i, j])
            except Exception:
                pass
    return df

In [ ]:
config_rows = str_to_df('''run_name ++ data_conf.num_names ++ data_conf.cat_cardinalities ++ data_conf.focus_on ++ evaluator.metrics ++ model.autoencoder.batch_transforms.set_features
amount/event_type ++ ['amount'] ++ [['event_type', 55]] ++ ['amount'] ++ [Reconstruction] ++ {"RemainOneFeatureToLearn: {"cat_features": [], "num_features": ['amount']}}
''', sep=' ++ ')
common_conf = {
    "runner.params.n_runs": 2,
    # "data_conf.focus_on": ["event_type"]
}
basic_conf = load_base_yaml("/home/dev/2025/transaction-generation/config.yaml")

{'config_factory': None, 'log_dir': 'log/generation/mbd', 'device': 'cuda:0', 'common_seed': 0, 'runner': {'name': 'GenerationTrainer', 'run_type': 'simple', 'seed_keys': ['common_seed'], 'params': {'n_runs': 1, 'n_workers': 1}, 'device_list': None}, 'logging': {'file_lvl': 'info', 'cons_lvl': 'info'}, 'trainer': {'total_iters': 1000000, 'total_epochs': None, 'ckpt_track_metric': 'Reconstruction overall', 'iters_per_epoch': None, 'patience': 10, 'profiling': False, 'verbose': True, 'ckpt_resume': None}, 'data_conf': {'cat_cardinalities': None, 'num_names': ['amount'], 'index_name': 'client_id', 'time_name': 'days_since_first_tx', 'focus_on': ['amount'], 'test_path': 'data/mbd-50k/test', 'train_path': 'data/mbd-50k/train', 'val_ratio': 0.15, 'num_workers': 4, 'train_resamples': 1, 'batch_size': 128, 'min_history_len': 32, 'generation_len': 32, 'max_seq_len': 1045, 'padding_value': 0, 'train_random_end': 'none', 'val_random_end': 'time', 'train_transforms': {'0': {'RescaleTime': {'loc': 

In [ ]:
generate_configs(config_rows, common_conf, basic_conf, output_dir="/home/dev/2025/transaction-generation/zhores/configs")


In [ ]:
config_rows = str_to_df('''run_name data_conf.num_names
amount/event_type   ['amount']
''')
common_conf = {
    "runner.params.n_runs": 2,
    # "data_conf.focus_on": ["event_type"]
}
basic_conf = load_base_yaml("/home/dev/2025/transaction-generation/config.yaml")
generate_configs(config_rows, common_conf, basic_conf, output_dir="/home/dev/2025/transaction-generation/zhores/configs")

KeyError: 'run_name'

In [5]:
# config_rows["run_name"] = config_rows["run_name"].str.replace("len", "len_old")
config_rows.loc[6] = {"run_name": "hist_len/full_train_33_val", "data_conf.max_seq_len": 33}

In [6]:
cols = ["test_Reconstruction overall", "test_Reconstruction overall_std", "test_Reconstruction event_type", "test_F1_macro on event_type", "test_F1_micro on event_type"]
print(df_to_str(collect_res(config_rows, cols)))

run_name	data_conf.max_seq_len	test_Reconstruction overall	test_Reconstruction overall_std	test_Reconstruction event_type	test_F1_macro on event_type	test_F1_micro on event_type
hist_len/33	33	-3,113789153160522	1,4176234054411647	0,4473422352922886	0,1884065007855574	0,4495562765005891
hist_len/64	64	-0,9192381663541508	0,203680866827878	0,474336566569265	0,1981738905368079	0,4804777592162543
hist_len/128	128	-2,8073141344591632	1,8230455736736508	0,4751193898854601	0,1987590799698307	0,4816197767114055
hist_len/256	256	0,5737644999737526	0,0614649497522162	0,5153446199496978	0,2131551317178197	0,5273745833186865
hist_len/512	512	0,5667375934681524	0,0401673176846223	0,5153073932242254	0,2125140534767633	0,5267226470536248
hist_len/1045	1045	0,5896790182171521	0,0078854769188361	0,5157323573062194	0,211917699329843	0,5262053940297504
hist_len/full_train_33_val	33	-3656,022326359088	0,0	0,439594475721323	0,181858267141067	0,4400439831104856



In [22]:
config_rows = str_to_df('''run_name	data_conf.max_seq_len	data_conf.train_transforms.shuffle.ShuffleBatch.untouched_slice	data_conf.val_transforms.shuffle.ShuffleBatch.untouched_slice	loss.name
event_type/gru_shuffle/no_shuffle	1045	[None, None]	[None, None]	baseline
event_type/gru_shuffle/test_hist_shuffle	1045	[None, None]	[-32, None]	baseline
event_type/gru_shuffle/basic_shuffle	1045	[0, None]	[-32, None]	baseline
event_type/gru_shuffle/no_shuffle_limit	64	[None, None]	[None, None]	baseline
event_type/gru_shuffle/test_hist_shuffle_limit	64	[None, None]	[-32, None]	baseline
event_type/gru_shuffle/basic_shuffle_limit	64	[0, None]	[-32, None]	baseline
event_type/gru_shuffle/head_shuffle_limit	64	[-32, None]	[-32, None]	tail
event_type/gru_shuffle/shuffle_pred	1045	[None, None]	[None, -32]	baseline
''')
common_conf = {
    "runner.params.n_runs": 2,
    "data_conf.focus_on": ["event_type"]
}
basic_conf = load_base_yaml("/home/transaction-generation/config.yaml")
generate_configs(config_rows, common_conf, basic_conf)

FileNotFoundError: [Errno 2] No such file or directory: '/home/transaction-generation/config.yaml'

In [9]:
config_rows.loc[8] = {"run_name": "event_type/gru_shuffle/shuffle_pred_real"}

In [10]:
cols = ["test_Reconstruction event_type", "test_Reconstruction event_type_std", "test_F1_macro on event_type", "test_F1_micro on event_type"]
print(df_to_str(collect_res(config_rows, cols)))

run_name	data_conf.max_seq_len	data_conf.train_transforms.shuffle.ShuffleBatch.untouched_slice	data_conf.val_transforms.shuffle.ShuffleBatch.untouched_slice	loss.name	test_Reconstruction event_type	test_Reconstruction event_type_std	test_F1_macro on event_type	test_F1_micro on event_type
event_type/gru_shuffle/no_shuffle	1045	[None, None]	[None, None]	baseline	0,4997503539599101	0,0079000682779505	0,2259678623416904	0,5316181120239498
event_type/gru_shuffle/test_hist_shuffle	1045	[None, None]	[-32, None]	baseline	0,4955031405214821	0,0073927074728879	0,2260842366954877	0,5288709338352093
event_type/gru_shuffle/basic_shuffle	1045	[0, None]	[-32, None]	baseline	0,5183340221872536	0,0007476104658076	0,207766155272697	0,522256871631362
event_type/gru_shuffle/no_shuffle_limit	64	[None, None]	[None, None]	baseline	0,5049566669448091	0,0037198618779548	0,2091703073653878	0,5137708164371042
event_type/gru_shuffle/test_hist_shuffle_limit	64	[None, None]	[-32, None]	baseline	0,5043174620323518	0

In [11]:
config_rows = str_to_df('''run_name	data_conf.max_seq_len	data_conf.train_transforms.shuffle.ShuffleBatch.untouched_slice	data_conf.val_transforms.shuffle.ShuffleBatch.untouched_slice	loss.name
gru_shuffle/no_shuffle	1045	[None, None]	[None, None]	baseline
gru_shuffle/test_hist_shuffle	1045	[None, None]	[-32, None]	baseline
gru_shuffle/basic_shuffle	1045	[0, None]	[-32, None]	baseline
gru_shuffle/no_shuffle_limit	64	[None, None]	[None, None]	baseline
gru_shuffle/test_shuffle_limit	64	[None, None]	[-32, None]	baseline
gru_shuffle/basic_shuffle_limit	64	[0, None]	[-32, None]	baseline
gru_shuffle/head_shuffle_limit	64	[-32, None]	[-32, None]	tail
gru_shuffle/shuffle_pred	1045	[None, None]	[None, -32]	baseline
''')
common_conf = {
    "runner.params.n_runs": 2,
}
basic_conf = load_base_yaml("/home/transaction-generation/config.yaml")
generate_configs(config_rows, common_conf, basic_conf)

sh transaction-generation/zhores/simple.sh gru_shuffle/no_shuffle
sh transaction-generation/zhores/simple.sh gru_shuffle/test_hist_shuffle
sh transaction-generation/zhores/simple.sh gru_shuffle/basic_shuffle
sh transaction-generation/zhores/simple.sh gru_shuffle/no_shuffle_limit
sh transaction-generation/zhores/simple.sh gru_shuffle/test_shuffle_limit
sh transaction-generation/zhores/simple.sh gru_shuffle/basic_shuffle_limit
sh transaction-generation/zhores/simple.sh gru_shuffle/head_shuffle_limit
sh transaction-generation/zhores/simple.sh gru_shuffle/shuffle_pred
✅ Generated 8 config files in '/home/transaction-generation/zhores/configs'
sh zhores/simple_mega.sh gru_shuffle/no_shuffle
sh zhores/simple_mega.sh gru_shuffle/test_hist_shuffle
sh zhores/simple_mega.sh gru_shuffle/basic_shuffle
sh zhores/simple_mega.sh gru_shuffle/no_shuffle_limit
sh zhores/simple_mega.sh gru_shuffle/test_shuffle_limit
sh zhores/simple_mega.sh gru_shuffle/basic_shuffle_limit
sh zhores/simple_mega.sh gru_shu

In [12]:
config_rows.loc[8] = {"run_name": "gru_shuffle/shuffle_pred_real"}

In [13]:
cols = ["test_Reconstruction overall", "test_Reconstruction overall_std", "test_Reconstruction event_type", "test_F1_macro on event_type", "test_F1_micro on event_type"]
print(df_to_str(collect_res(config_rows, cols)))

run_name	data_conf.max_seq_len	data_conf.train_transforms.shuffle.ShuffleBatch.untouched_slice	data_conf.val_transforms.shuffle.ShuffleBatch.untouched_slice	loss.name	test_Reconstruction overall	test_Reconstruction overall_std	test_Reconstruction event_type	test_F1_macro on event_type	test_F1_micro on event_type
gru_shuffle/no_shuffle	1045	[None, None]	[None, None]	baseline	0,626950538500848	0,0348298597198615	0,5165842942072534	0,2141471924652556	0,529716285984141
gru_shuffle/test_hist_shuffle	1045	[None, None]	[-32, None]	baseline	0,398332970957412	0,2377677390287732	0,5119249446544822	0,2127402166075434	0,5248385525745295
gru_shuffle/basic_shuffle	1045	[0, None]	[-32, None]	baseline	0,4147420273943289	0,1587370093397339	0,514614198738848	0,2065546753726739	0,5184586539801697
gru_shuffle/no_shuffle_limit	64	[None, None]	[None, None]	baseline	-0,3827377440691157	0,3215921002042922	0,4923006256947674	0,2050625980120491	0,5004751565442429
gru_shuffle/test_shuffle_limit	64	[None, None]	[